In [1]:
import pandas as pd
import cohere
import dvc.api as DvcApi
from dotenv import load_dotenv 

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [2]:
import sys, os

sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("../scripts")))

In [3]:
import prompt_functions as promptf

In [4]:
#load_dotenv(encoding='utf-16')

#api_key = os.getenv('apikey')

In [5]:
# Create and retrieve a Cohere API key from os.cohere.ai
# co = cohere.Client(api_key)

In [6]:
#pd.set_option('display.max_colwidth', None)

# Project1

### Project Overview
A client has a system that collects news artifacts from web pages, tweets, facebook posts, etc. The client is interested in scoring a given new artifact against a topic. The client has hired experts to score a few of these news items. The range of results between 0 and 10 signifies the degree of relevance of the news item to the topic (breaking news that may lead to **public unrest**)

We want to explore the efficiency of GPT3-like LLMs to this task. If the recommendation is positive, we must demonstrate that our strategies to design prompts are reproducible and produce a consistent result. 

Design a pipeline that takes a news item (e.g. title +  description + body) and returns a score for the news item

The columns of this data are as follows

- Domain: the base URL or a reference to the source these item comes from 
- **Title:** title of the item - the content of the item
- **Description:** the content of the item
- **Body:** the content of the item
- Link: URL to the item source (it may not functional anymore sometime)
- Timestamp: timestamp that this item was collected at
- **Analyst_Average_Score:** target variable - the score to be estimated 
- Analyst_Rank: score as rank
- Reference_Final_Score: Not relevant for now - it is a transformed quantity

In [4]:
path = "data/Example_data1.csv"
repo = "../"
version = "v0"

data_url = DvcApi.get_url(path = path, repo = repo, rev = version) #could be tag or git commit
data_news = pd.read_csv(data_url)

In [5]:
data_news.head(2)

,Domain,Title,Description,Body,Link,timestamp,Analyst_Average_Score,Analyst_Rank,Reference_Final_Score
0,rassegnastampa.news,Boris Johnson using a taxpayer-funded jet for ...,…often trigger a protest vote that can upset…t...,Boris Johnson using a taxpayer-funded jet for ...,https://rassegnastampa.news/boris-johnson-usin...,2021-09-09T18:17:46.258006,0.0,4,1.96
1,twitter.com,"Stumbled across an interesting case, a woman f...","Stumbled across an interesting case, a woman f...","Stumbled across an interesting case, a woman f...",http://twitter.com/CoruscaKhaya/status/1435585...,2021-09-08T13:02:45.802298,0.0,4,12.00


In [6]:
data_news['Analyst_Average_Score']

0    0.00
1    0.00
2    0.00
3    0.00
4    0.00
5    1.33
6    0.00
7    1.66
8    0.33
9    0.00
Name: Analyst_Average_Score, dtype: float64

We have a very small sample, 

- Ten examples in total,
- only three have non-zero relevance to public unrest.
- all values are below score of 2


In [8]:
df = pd.DataFrame(columns = ['text', 'Avg_Score','label'], index=range(10))

df['text'] = data_news['Title'] + data_news['Body']
df['Avg_Score']= data_news['Analyst_Average_Score']
df['label'] = data_news['Analyst_Average_Score'].apply(lambda x: 'Relevant' if x > 0 else 'Irrelevant')

df.head(3)

,text,Avg_Score,label
0,Boris Johnson using a taxpayer-funded jet for ...,0.0,Irrelevant
1,"Stumbled across an interesting case, a woman f...",0.0,Irrelevant
2,Marché Résines dans les peintures et revêtemen...,0.0,Irrelevant
